## Python Screening Assignment


   1. Create a function in python to read the text file and replace specific content of the file.

        File name: example.txt
        
        Origin file content: This is a placement assignment
        
        Replace string:  Placement should be replaced by screening.
        
        Replaced file content: This is a screening assignment

2. Demonstrate use of abstract class, multiple inheritance and decorator in python using examples.
    

In [1]:
import re
  
def python_fileupdate(search_text='placement',replace_text='screening', file_name='example.txt'):

    try:
        with open(file_name,'r+') as f:
            file = f.read() # Reading the file data 
            file= file.lower()
            file = re.sub(search_text, replace_text, file)
            f.seek(0)# seek the pointer to beginning
            f.write(file)
            f.truncate()
    except IOError:
        print("Error: Could not find file or read data")
    else: 
        print("Content updated  successfully")
        f.close()

In [2]:
python_fileupdate()

Content updated  successfully


## Using Abstract class with MLModel

In this code I am creating a MLModel abstract class with inputSchema (to define data take input for prediction) output schema (what output get for display or API to digest as JSON or other format) and a predict method.

Them we create a simple train method to train the model with iris dataset and then we create a child class ***IrisSVCModel(MLModel)*** and implement all three abstract method to load the trained model and done prediction with defined input/output schema. Then we create a simple predict method to return output schema for API digestion. Rest of the code is self-explanatory.

Note: I have went thro different blog posts to comiple this code.


In [4]:
from abc import ABC, abstractmethod


class MLModel(ABC):
    
    @property
    @abstractmethod
    def input_schema(self):
        raise NotImplementedError()

    @property
    @abstractmethod
    def output_schema(self):
        raise NotImplementedError()

    @abstractmethod
    def __init__(self):
        raise NotImplementedError()

    @abstractmethod
    def predict(self, data):
        self.input_schema.validate(data)

In [5]:
from sklearn import datasets
from sklearn import svm
import pickle
import os


def train():
    #load data from IRIS data set
    iris = datasets.load_iris()
    #create a model
    svm_model = svm.SVC(gamma=0.001, C=100.0)
    #fit the model with IRIS Dataset
    svm_model.fit(iris.data[:-1], iris.target[:-1])

    #create a pickle file in local directory
    file = open("svc_model.pickle", 'wb')
    #dump the file
    pickle.dump(svm_model,file)
    print("Training completed and {} is saved".format(file.name))
    #close the file to access by predict method
    file.close()

In [6]:
train()

Training completed and svc_model.pickle is saved


In [21]:
import os
import pickle
from schema import Schema, Or
from numpy import array

class IrisSVCModel(MLModel):
    """ A demonstration of how to use  """
    input_schema = Schema({'sepal_length': float,
                           'sepal_width': float,
                           'petal_length': float,
                           'petal_width': float})

    # the output of the model will be one of three strings
    output_schema = Schema({'species': Or("setosa", "versicolor", "virginica")})

    def __init__(self):
        file = open("svc_model.pickle", 'rb')
        self._svm_model = pickle.load(file)
        file.close()

    def predict(self, data):
        # calling the super method to validate against the input_schema
        super().predict(data=data)

        # converting the incoming dictionary into a numpy array that can be accepted by the  model
        X = array([data["sepal_length"], data["sepal_width"], data["petal_length"], data["petal_width"]]).reshape(1, -1)

        # making the prediction and extracting the result from the array
        y_hat = int(self._svm_model.predict(X)[0])

        #converting the prediction into a string that will match the output schema of the model
        # this list will map the output of the scikit-learn model to the output string expected by the schema
        targets = ['setosa', 'versicolor', 'virginica']
        species = targets[y_hat]

        return {"species": species}

In [68]:
data = {'sepal_length': 1.0,
        'sepal_width': 5.0,
        'petal_length': 1.0,
        'petal_width': 1.0}

In [69]:
pred= IrisSVCModel()
pred.predict(data)

{'species': 'setosa'}

# Multiple inheritence

A class can be derived from more than one base class in Python, similar to C++. This is called multiple inheritance. In multiple inheritance, the features of all the base classes are inherited into the derived class. 

In [78]:
#create a Tokenizer class who splits the sentence into tokens
class Tokenizer:
    def __init__(self, text):
        print('Tokenizer Class')
        self.tokens = text.split()

        
# WordCounter count no of tokens by inherit the Tokenizer class
class WordCounter(Tokenizer):
    def __init__(self, text):
        print('Start WordCounter class')
        super().__init__(text)
        self.word_count = len(self.tokens)

# Vocabulary class Find unique words in text
class Vocabulary(Tokenizer):
    def __init__(self, text):
        print('Start Vocabulary class')
        super().__init__(text)
        self.vocab = set(self.tokens)

#TextDescribes class inherit both Word count and Vocabulary class which shows multiple inheritence     
class TextDescriber(WordCounter, Vocabulary):
    def __init__(self, text):
        print('Start TextDescriber class')
        super().__init__(text)


In [79]:
td = TextDescriber('Hi I am student of Ineuron, I am learning Data Science, I am from Delhi')

Start TextDescriber class
Start WordCounter class
Start Vocabulary class
Tokenizer Class


In [82]:
print("TOKEN: ", td.tokens)
print("VOCAB: ", td.vocab)
print("Word Length: ", td.word_count)

TOKEN:  ['Hi', 'I', 'am', 'student', 'of', 'Ineuron,', 'I', 'am', 'learning', 'Data', 'Science,', 'I', 'am', 'from', 'Delhi']
VOCAB:  {'learning', 'of', 'Science,', 'Data', 'student', 'from', 'Ineuron,', 'am', 'Delhi', 'Hi', 'I'}
Word Length:  15


# Decorator 
A decorator in Python is a function that takes another function as an argument and extends its behavior without explicitly modifying it. For example from Krish Sir vedio, in the following code we can create main_welcome function which can take a function as arguments and decorate it with sub_welcome_class, we can use the same logic for logging in Data science model building.

In [85]:
### Decorators
def main_welcome(func):
    def sub_welcome_class():
        print("Welcome To Krish Naik Youtube Channel")
        func()
        print("Please subscribe Krish channel")
    return sub_welcome_class()

In [86]:
@main_welcome
def channel_name():
    print("This is Krish youtube channel")

Welcome To Krish Naik Youtube Channel
This is Krish youtube channel
Please subscribe Krish channel


By using the same logic we can create a ***log_computation*** function which takes function as argument and define a wrapper function which can be used to log the info from which function it is called and result in the log file.

In [108]:
import logging

def log_computation(func):
    def wrapper(*args):
        logging.basicConfig(filename='Logger.log',level=logging.INFO)
        logging.info("Info message being logged from function: " + str(func))
        res = func(*args)
        logging.info("The result from function call is: "+ str(res))
        return res
    return wrapper

In [109]:
@log_computation
def sum_up(n, m):
    return n + m

In [110]:
sum_up(3,4)

7

In [111]:
@log_computation
def substract(n, m):
    return n - m

In [112]:
substract(10,3)

7